# Gradient Descent for Linear Regression

You will:

 * Update gradient descent for logistic regression.
 * See gradient descent on a familiar data set.

### Logistic Gradient Descent

Recall the gradient descent algorithm utilizes the gradient calculation:

$$\begin{align*}
&\text{repeat until convergence:} \; \lbrace \\
&  \; \; \;w_j = w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{1}  \; & \text{for j := 0..n-1} \\ 
&  \; \; \;  \; \;b = b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b} \\
&\rbrace
\end{align*}$$

Where each iteration performs simultaneous updates on $w_j$ for all $j$, where
$$\begin{align*}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)} \tag{2} \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \tag{3} 
\end{align*}$$

* m is the number of training examples in the data set      
* $f_{\mathbf{w},b}(x^{(i)})$ is the model's prediction, while $y^{(i)}$ is the target
* For a logistic regression model  
    $z = \mathbf{w} \cdot \mathbf{x} + b$  
    $f_{\mathbf{w},b}(x) = g(z)$  
    where $g(z)$ is the sigmoid function:  
    $g(z) = \frac{1}{1+e^{-z}}$ 

#### Calculating the Gradient, Code Description
Implements equation (2),(3) above for all $w_j$ and $b$.
There are many ways to implement this. Outlined below is this:
- initialize variables to accumulate `dj_dw` and `dj_db`
- for each example
    - calculate the error for that example $g(\mathbf{w} \cdot \mathbf{x}^{(i)} + b) - \mathbf{y}^{(i)}$
    - for each input value $x_{j}^{(i)}$ in this example,  
        - multiply the error by the input  $x_{j}^{(i)}$, and add to the corresponding element of `dj_dw`. (equation 2 above)
    - add the error to `dj_db` (equation 3 above)

- divide `dj_db` and `dj_dw` by total number of examples (m)
- note that $\mathbf{x}^{(i)}$ in numpy `X[i,:]` or `X[i]`  and $x_{j}^{(i)}$ is `X[i,j]`

**Python - imports**

**C - imports**

In [12]:
import numpy as np
import matplotlib.pyplot as plt
import math, copy

**Python - sigmoid**

**C - sigmoid**

In [2]:
def sigmoid(z):
    """
    compute sigmoid

    Args:
        z (scalar):  logistic function, f_wb

    Return:
        g (scalar):  sigmoid of f_wb
    """
    return 1/(1 + np.exp(-z))

**Python - gradient**

**C - gradient**

In [3]:
def compute_gradient_logistic(X, y, w, b):
    """
    Computes the gradient for linear regression

    Args:
        X (adarray (m,n)):  Data, m examples with n features
        y (ndarray (m, )):  target values
        w (ndarray (n, )):  model parameters
        b (scalar)       :  model parameter

    Returns:
        dj_dw (ndarray (n, )):  The gradient of the cost w.r.t the parameters w.
        dj_db (scalar)       :  The gradient of the cost w.r.t the parameter b.
    """

    m,n = X.shape
    dj_dw = np.zeros((n,))              #(n,)
    dj_db = 0.

    for i in range(m):
        f_wb_i = sigmoid(np.dot(X[i], w) + b)           #(m,n)(n,)=scalar
        err_i = f_wb_i - y[i]                           #scalar
        for j in range(n):
            dj_dw[j] = dj_dw[j] + err_i * X[i,j]        #scalar
        dj_db = dj_db + err_i
    dj_dw = dj_dw/m                                     #(n,)
    dj_db = dj_db/m                                     #scalar

    return dj_dw,dj_db

**Python - cost**

**C - cost**

In [4]:
def compute_cost_logistic(X, y, w, b):
    """
    Compute cost
    Args:
        X (ndarray (m,n)):  input data 
        y (ndarray (m,)):   output data
        w (ndarray (n,)):   model parametrs 
        b (scalar):         model parameter

    Return:
        J (sclar):  Cost of logistic model
    """

    J = 0.0
    f_wb_i = 0.0
    m = X.shape[0]

    for i in range(m):
        f_wb_i = sigmoid(np.dot(X[i], w) + b)                             #(m,): scalar
        J += -y[i]*np.log(f_wb_i) - (1 - y[i])*np.log(1 - f_wb_i)   
    J /= m
    
    return J

**Python - main**

**C - main**

In [5]:
def main():
    X_tmp = np.array([[0.5, 1.5], [1,1], [1.5, 0.5], [3, 0.5], [2, 2], [1, 2.5]])
    y_tmp = np.array([0, 0, 0, 1, 1, 1])
    w_tmp = np.array([2.,3.])
    b_tmp = 1.

    dj_dw_tmp, dj_db_tmp = compute_gradient_logistic(X_tmp, y_tmp, w_tmp, b_tmp)

    print(f"dj_dw: {dj_dw_tmp}")
    print(f"dj_db: {dj_db_tmp}")


if __name__=="__main__":
    main()

dj_dw: [0.49833339 0.49883943]
dj_db: 0.49861806546328574


**Python - Gradient Descent**

**C - Gradient Descent**

In [6]:
def gradient_descent(X, y, w_in, b_in, alpha, num_iter):
    """
    Performs batch gradient descent
    
     Args:
      X (ndarray (m,n)   : Data, m examples with n features
      y (ndarray (m,))   : target values
      w_in (ndarray (n,)): Initial values of model parameters  
      b_in (scalar)      : Initial values of model parameter
      alpha (float)      : Learning rate
      num_iters (scalar) : number of iterations to run gradient descent
      
    Returns:
      w (ndarray (n,))   : Updated values of parameters
      b (scalar)         : Updated value of parameter 
    """

    #An array to store cost J and w's each iteration primarily for graphing later
    J_history = []
    w = copy.deepcopy(w_in)     #avoid modifying global w within function
    b = b_in

    for i in range(num_iter):
        #calculate gradient and update the parameters
        dj_dw, dj_db = compute_gradient_logistic(X, y, w, b)

        #Update Parameters using w, b, alpha and gardient
        w = w - alpha * dj_dw
        b = b - alpha * dj_db

        #Save cost J each iteration
        if i<100000:          #prevent resource exhaustion
            J_history.append(compute_cost_logistic(X, y, w, b))

        #Print cost every at intervals 10 times or as many iterations if < 10
        if i% math.ceil(num_iter / 10) ==0:
          print(f"Iteration {i:4d}  Cost {J_history[-1]}  ")

    return w, b, J_history                 #return final w, b and J history for graphing

In [15]:
def main():
    X_train = np.array([[0.5, 1.5], [1,1], [1.5, 0.5], [3, 0.5], [2, 2], [1, 2.5]])
    y_train = np.array([0, 0, 0, 1, 1, 1])
    
    w_tmp  = np.zeros_like(X_train[0])
    b_tmp  = 0.
    alph = 0.1
    iters = 100000

    w_out, b_out, _=gradient_descent(X_train, y_train, w_tmp, b_tmp, alph, iters)
    print(f"\nupdated parameters: w:{w_out},    b:{b_out}")

if __name__=="__main__":
    main()

Iteration    0  Cost 0.684610468560574  
Iteration 10000  Cost 0.01711604647887364  
Iteration 20000  Cost 0.008523403979166467  
Iteration 30000  Cost 0.005672197191107633  
Iteration 40000  Cost 0.004250161053834308  
Iteration 50000  Cost 0.003398230224179212  
Iteration 60000  Cost 0.0028308425601004327  
Iteration 70000  Cost 0.002425848306579758  
Iteration 80000  Cost 0.0021222573122028584  
Iteration 90000  Cost 0.0018862216652143864  

updated parameters: w:[8.35313087 8.15226727],    b:-22.690605796630248
